In [ ]:
# Please Install the following Libraries in order to make the widget work in your notebook
!pip install scikit-surprise
!pip install --upgrade setuptools
!pip install plotly
!pip install webshot
!pip install phantomjs
!pip install pandasql
!pip install ipywidgets

In [1]:
import pandas as pd
import numpy as np
import pandasql as ps
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#from ipywidgets import widgets

# Data Processing

In [2]:
campaigns = pd.read_csv("campaigns_202106162119.csv")

users = pd.read_csv("users_202106162127.csv")

users['quantile_rank'] = pd.qcut(users['attended'], 4,
                               labels = False)

output_df  = ps.sqldf("""
                        select 
u.user_id ,
country,
section,
case 
	when source like '%.%' then 'domains'
	when source like '%google%' then 'google'
	when source like '%outube%' then 'youtube'
	when source like '%nstagram%' then 'instagram'
	when source in (select source from campaigns group by 1 having count(*) < 10) then 'others'
	else source
	end as source,
	medium,
	u.attended ,
	u.booked ,
case 
    when quantile_rank = 0 then 'Very low'
    when quantile_rank = 1 then 'Low'
    when quantile_rank = 2 then 'Medium'
    when quantile_rank = 3 then 'High'
    end as "Activity Level"
from users u 
join campaigns c
on u.user_id = c.user_id
group by 1,2,3,4,5,6,7,8
                        """)

output_df.head(10)

,user_id,country,section,source,medium,attended,booked,Activity Level
0,702117,USA,french,google,cpc,3,4,Very low
1,702540,France,english,blog,blog,11,30,Low
2,702540,France,english,direct,direct,11,30,Low
3,702540,France,english,domains,referral,11,30,Low
4,702540,France,english,facebook,cpc,11,30,Low
5,702540,France,english,google,cpc,11,30,Low
6,702540,France,english,google,organic,11,30,Low
7,702540,France,english,iterable,email,11,30,Low
8,702540,France,english,others,social,11,30,Low
9,702540,France,english,youtube,social,11,30,Low


In [5]:
active_section_analysis=ps.sqldf("""select country,section,count(*) as no_of_users
         from output_df where "Activity Level" = "High" group by 1,2""")

inactive_section_analysis=ps.sqldf("""select country,section,count(*) as no_of_users
         from output_df where "Activity Level" = "Very low" group by 1,2""")

x=active_section_analysis['country'].unique()
fig = make_subplots(rows=1, cols=2, shared_yaxes=False,)

fig.add_trace(go.Bar(x=x, y=active_section_analysis[active_section_analysis['section'] == 'english']["no_of_users"], name='english',  marker_color='#1f77b4'),1,1)
fig.add_trace(go.Bar(x=x, y=active_section_analysis[active_section_analysis['section'] == 'spanish']["no_of_users"], name='spanish',  marker_color='#e377c2'),1,1)
fig.add_trace(go.Bar(x=x, y=active_section_analysis[active_section_analysis['section'] == 'german']["no_of_users"], name='german',  marker_color='#20b2aa'),1,1)
fig.add_trace(go.Bar(x=x, y=active_section_analysis[active_section_analysis['section'] == 'french']["no_of_users"], name='french',  marker_color='#9467bd'),1,1)

x=inactive_section_analysis['country'].unique()
fig.add_trace(go.Bar(x=x, y=inactive_section_analysis[inactive_section_analysis['section'] == 'english']["no_of_users"], name='english',marker_color='#1f77b4',showlegend=False),1,2)
fig.add_trace(go.Bar(x=x, y=inactive_section_analysis[inactive_section_analysis['section'] == 'spanish']["no_of_users"], name='spanish',marker_color='#e377c2',showlegend=False),1,2)
fig.add_trace(go.Bar(x=x, y=inactive_section_analysis[inactive_section_analysis['section'] == 'german']["no_of_users"], name='german',marker_color='#20b2aa',showlegend=False),1,2)
fig.add_trace(go.Bar(x=x, y=inactive_section_analysis[inactive_section_analysis['section'] == 'french']["no_of_users"], name='french',marker_color='#9467bd',showlegend=False),1,2)


fig.update_layout(title="Section Distribution: High Activity Users vs Low Activity Users",barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':inactive_section_analysis['country'].unique()})
fig.show()